In [2]:
import os

In [3]:
os.chdir("../")

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    max_features: str
    max_leaf_nodes: int
    min_impurity_decrease: float
    min_samples_leaf: int
    min_samples_split: int
    splitter: str
    target_column: str

In [28]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.DecisionTreeRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            max_depth =  params.max_depth,
            max_features =  params.max_features,
            max_leaf_nodes =  params.max_leaf_nodes,
            min_impurity_decrease =   params.min_impurity_decrease,
            min_samples_leaf =  params.min_samples_leaf,
            min_samples_split =  params.min_samples_split,
            splitter =  params.splitter,
            target_column = schema.name
            
        )

        return model_trainer_config

In [30]:
import pandas as pd
import os
from mlProject import logger
from sklearn.tree import DecisionTreeRegressor
import joblib

In [31]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = DecisionTreeRegressor(max_depth=self.config.max_depth,max_features=self.config.max_features,max_leaf_nodes=self.config.max_leaf_nodes,min_impurity_decrease=self.config.min_impurity_decrease,
                                   min_samples_leaf=self.config.min_samples_leaf,min_samples_split=self.config.min_samples_split,splitter=self.config.splitter,random_state=42)        
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [41]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-08-09 15:08:58,161: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-09 15:08:58,162: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-09 15:08:58,165: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-09 15:08:58,166: INFO: common: created directory at: artifacts]
[2023-08-09 15:08:58,167: INFO: common: created directory at: artifacts/model_trainer]
